In [1]:
import pandas as pd

# 데이터 불러오기
vehicle = pd.read_csv('Vehicle.csv')
order = pd.read_csv('Order_Monday.csv')
stop = pd.read_csv('Stop.csv')
center = pd.read_csv('Center.csv')
od_matrix = pd.read_csv('OD_Matrix.txt', delimiter=',')

# 데이터프레임의 숫자 열을 double 형식으로 변환
def convert_to_float(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')
    return df

vehicle = convert_to_float(vehicle)
order = convert_to_float(order)
stop = convert_to_float(stop)
center = convert_to_float(center)
od_matrix = convert_to_float(od_matrix)

# Terminal ID -> S_0
stop.replace('Terminal', 'S_0', inplace=True)
vehicle.replace('Terminal', 'S_0', inplace=True)
order.replace('Terminal', 'S_0', inplace=True)
stop.replace('Terminal', 'S_0', inplace=True)
center.replace('Terminal', 'S_0', inplace=True)
od_matrix.replace('Terminal', 'S_0', inplace=True)


# parameter 값 저장
# vehicle
num_vehicle_types = 5
Vton = vehicle['VehTon'].values
Vstart = vehicle['BusinessStartTM'].values
Vend = vehicle['BusinessEndTM'].values
fixed_cost = vehicle['FixedCost'].values
round_cost = vehicle['RoundCost'].values
variable_cost = vehicle['VariableCost'].values
maxcapa_plt = vehicle['MaxCapaPLT'].values
maxcapa_weight = vehicle['MaxCapaWeight'].values
max_rounds = [int(vehicle['MaxRound'].values[i]) for i in range(num_vehicle_types)]
max_count = vehicle['MaxCount'].values
# center
center_y = center['Y'].iloc[0]
center_x = center['X'].iloc[0]
center_id = center['ID'].iloc[0]
center_start = center['Start_time'].iloc[0]
center_end = center['End_time'].iloc[0]
center_load_time = 30  # 로드 시간 30분
# order
order_id = order['Order_ID'].str.replace('O_', '').astype(int).values
order_y = order['Y'].values
order_x = order['X'].values
order_stop_id = order['Stop_ID'].str.replace('S_', '').astype(int).values
order_plt = order['plt'].values
order_weight = order['weight'].values
order_last = int(order_id[-1])
num_orders = len(order_id)
# stop
stop_id = stop['ID'].str.replace('S_', '').astype(int).values
stop_y = stop['Y'].values
stop_x = stop['X'].values
stop_start = stop['Start_time'].values
stop_end = stop['End_time'].values
stop_access = stop['Access_restriction'].values
stop_unload = stop['Unloading_time'].values
stop_unloadplt = stop['Unloading_time_per_plt'].values
stop_last = int(stop_id[-1])
num_stops = len(stop_id)

# OD Matrix 데이터 준비
od_origin_id = od_matrix['OriginID'].apply(lambda x: 0 if x == 'Terminal' else int(''.join(filter(str.isdigit, x)))).values
od_destination_id = od_matrix['DestinationID'].apply(lambda x: 0 if x == 'Terminal' else int(''.join(filter(str.isdigit, x)))).values
od_time = od_matrix['Total_Time'].astype(float).values
od_distance = od_matrix['Total_Distance'].astype(float).values



print(f'stop_last = {stop_last}')
print(f'num_stops = {num_stops}')
print(f'order_last = {order_last}')
print(f'num_orders = {num_orders}')
print(f'stop_id & order_id & order_stop_id  type은 {type(stop_id[0])} & {type(order_id[0])} & {type(order_stop_id[0])}')
print(type(max_count[2]))


print(len(od_origin_id))
print(len(od_destination_id))
print(od_destination_id[1])
print(len(od_time))

print(order_stop_id[655])


stop_last = 1127
num_stops = 1092
order_last = 1990
num_orders = 1990
stop_id & order_id & order_stop_id  type은 <class 'numpy.int32'> & <class 'numpy.int32'> & <class 'numpy.int32'>
<class 'numpy.float64'>
1192466
1192466
4
1192466
1094


In [2]:
from ortools.linear_solver import pywraplp
import time

print('start')
t1 = time.time()

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')


##dummy data 대신 들어가야할 것

num_vehicles = 20

V = {}
Vround = {}
Vdistance = {}
Vplt = {}
Vcarry = {}
visit = {}
Tarrive = {}
Tdepart = {}

T = []
T.append(time.time())

for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        Vround_name = f'Vround_{i}_{j}'
        Vdistance_name = f'Vdistance_{i}_{j}'
        Vplt_name = f'Vplt_{i}_{j}'
        Vcarry_name = f'Vcarry_{i}_{j}'
        
        Vround[i, j] = solver.IntVar(0, max_rounds[i], Vround_name)
        Vdistance[i, j] = solver.NumVar(0, solver.infinity(), Vdistance_name)
        Vplt[i, j] = solver.NumVar(0, maxcapa_plt[i], Vplt_name)
        Vcarry[i, j] = solver.NumVar(0, maxcapa_weight[i], Vcarry_name)

        for r in range(max_rounds[i]):
            for k in range(num_orders):
                V_name = f'V_{i}_{j}_{r}_{k}'
                Tarrive_name = f'Tarrive_{i}_{j}_{r}_{k}'
                Tdepart_name = f'Tdepart_{i}_{j}_{r}_{k}'

                V[i, j, r, k] = solver.BoolVar(V_name)
                Tarrive[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tarrive_name)
                Tdepart[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tdepart_name)
            
            for h in range(num_orders): #terminal 떄문에
                visit_name = f'visit_{i}_{j}_{r}_{h}'
                visit[i, j, r, h] = solver.BoolVar(visit_name)
                
    #time for each i
    T.append(time.time())
    print(f'something happend 0-({i}) {round(T[i+1]-T[i],3)}s')

#dict size check
for i in range(8):
    dict = [V, Vround, Vdistance, Vplt, Vcarry, visit, Tarrive, Tdepart]
    dict_name = ['V', 'Vround', 'Vdistance', 'Vplt', 'Vcarry', 'visit', 'Tarrive', 'Tdepart']
    print(f'length of {dict_name[i]}: {len(dict[i])}')
    
t2 = time.time()
print(f't1~t2: {round(t2-t1,3)}s')
T.append(t2)

start
something happend 0-(0) 1.148s
something happend 0-(1) 1.065s
something happend 0-(2) 1.259s
something happend 0-(3) 1.2s
something happend 0-(4) 1.376s
length of V: 398000
length of Vround: 100
length of Vdistance: 100
length of Vplt: 100
length of Vcarry: 100
length of visit: 398000
length of Tarrive: 398000
length of Tdepart: 398000
t1~t2: 6.081s


In [3]:
# 제약식 추가
M = 100000  # Sufficiently large number

T=[t2]

for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            # constraint1: V_ijrk의 순서를 보장하는 제약식
            if j < num_vehicles - 1:  # j+1 때문에 num_vehicles-1까지만
                lhs = solver.Sum(V[i, j+1, r, k] for k in range(num_orders))
                rhs = solver.Sum(V[i, j, r, k] for k in range(num_orders))
                solver.Add(lhs <= rhs)

            # constraint2: Vround_ij, Vdistance_ij, Vplt_ij, Vcarry_ij 값이 0보다 큰 경우
            total_visits = solver.Sum(V[i, j, r, k] for k in range(order_last))
            solver.Add(Vround[i, j] <= M * total_visits)
            solver.Add(Vdistance[i, j] <= M * total_visits)
            solver.Add(Vplt[i, j] <= M * total_visits)
            solver.Add(Vcarry[i, j] <= M * total_visits)

            # constraint3: 최대 회전 수 제한
            solver.Add(Vround[i, j] <= max_rounds[i])
            
            # constraint4: 최대 stop 수, stop 방문 제약
            for k in range(1,num_orders+1):    #terminal id 때문에 1부터 num_orders+1까지
                h_k = order_stop_id[k-1]             
                solver.Add(V[i, j, r, k-1] == visit[i, j, r, h_k])
            solver.Add(sum(visit[i, j, r, h] for h in range(num_stops)) <= max_count[i])
            
             # constraint5: 최대 적재량 및 적재 무게 제한
            solver.Add(solver.Sum(V[i, j, r, k] * order_plt[k] for k in range(order_last)) <= maxcapa_plt[i])
            solver.Add(solver.Sum(V[i, j, r, k] * order_weight[k] for k in range(order_last)) <= maxcapa_weight[i])
            
             # constraint6: 접근 제한
            for h in range(num_stops):
                solver.Add(visit[i, j, r, h] * Vton[i] <= stop_access[h])
                
            # 일곱 번째 제약식: 시간 제약
            # TSP 제약식

    T.append(time.time())
    print(f'something happend in constraints-({i}) {round(T[i+1]-T[i],3)}s')
    
total_cost = solver.Sum([fixed_cost[i] + Vround[i, j] * round_cost[i] + Vdistance[i, j] * variable_cost[i]
                         for i in range(num_vehicle_types) for j in range(num_vehicles)]) \
             + solver.Sum(V[i, j, r, k] for i in range(num_vehicle_types) for j in range(num_vehicles) for r in range(max_rounds[i]) for k in range(num_orders))

solver.Minimize(total_cost)
# Solve the problem
status = solver.Solve()

# Check the result
if status == pywraplp.Solver.OPTIMAL:
    print(f'Total cost = {solver.Objective().Value()}')
    for i in range(num_vehicle_types):
        for j in range(num_vehicles):
            for r in range(max_rounds[i]):
                for k in range(num_orders):
                    if V[i, j, r, k].solution_value():
                        print(f'Vehicle {i}_{j} round {r} serves order {k}')
else:
    print('No optimal solution found.')

T_final = time.time()
print(f'Total time: {round(T_final - T[0], 3)}s')

print("all alright")


something happend in constraints-(0) 9.726s
something happend in constraints-(1) 5.197s
something happend in constraints-(2) 4.489s
something happend in constraints-(3) 6.602s
something happend in constraints-(4) 6.905s
Total cost = 18756000.0
Total time: 50.759s
all alright


In [6]:
from ortools.linear_solver import pywraplp
import time

t1 = time.time()
T = []
print('pass')

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')


##dummy data 대신 들어가야할 것

num_vehicles = 20

V = {}
Vround = {}
Vdistance = {}
Vplt = {}
Vcarry = {}
visit = {}
Tarrive = {}
Tdepart = {}


T.append(time.time())

for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        Vround_name = f'Vround_{i}_{j}'
        Vdistance_name = f'Vdistance_{i}_{j}'
        Vplt_name = f'Vplt_{i}_{j}'
        Vcarry_name = f'Vcarry_{i}_{j}'
        
        Vround[i, j] = solver.IntVar(0, max_rounds[i], Vround_name)
        Vdistance[i, j] = solver.NumVar(0, solver.infinity(), Vdistance_name)
        Vplt[i, j] = solver.NumVar(0, maxcapa_plt[i], Vplt_name)
        Vcarry[i, j] = solver.NumVar(0, maxcapa_weight[i], Vcarry_name)

        for r in range(max_rounds[i]):
            for k in range(order_last):
                V_name = f'V_{i}_{j}_{r}_{k}'
                Tarrive_name = f'Tarrive_{i}_{j}_{r}_{k}'
                Tdepart_name = f'Tdepart_{i}_{j}_{r}_{k}'

                V[i, j, r, k] = solver.BoolVar(V_name)
                Tarrive[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tarrive_name)
                Tdepart[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tdepart_name)
            
            for h in range(len(stop_id)):
                visit_name = f'visit_{i}_{j}_{r}_{h}'
                visit[i, j, r, str(h)] = solver.BoolVar(visit_name)
    T.append(time.time())
    print(f'something happend 0-({i}) {T[i+1]-T[i]}')

    
    
t2 = time.time()
print('t1~t2: ',t2-t1)
T.append(t2)

# 제약식 추가
M = 100000  # Sufficiently large number

for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            # 첫 번째 제약식: V_ijrk의 순서를 보장하는 제약식
            if j < num_vehicles - 1:  # j+1 때문에 num_vehicles-1까지만
                lhs = solver.Sum(V[i, j+1, r, k] for k in range(order_last))
                rhs = solver.Sum(V[i, j, r, k] for k in range(order_last))
                solver.Add(lhs <= rhs)

            # 두 번째 제약식: Vround_ij, Vdistance_ij, Vplt_ij, Vcarry_ij 값이 0보다 큰 경우
            total_visits = solver.Sum(V[i, j, r, k] for k in range(order_last))
            solver.Add(Vround[i, j] <= M * total_visits)
            solver.Add(Vdistance[i, j] <= M * total_visits)
            solver.Add(Vplt[i, j] <= M * total_visits)
            solver.Add(Vcarry[i, j] <= M * total_visits)

            # 세 번째 제약식: 최대 회전 수 제한
            solver.Add(Vround[i, j] <= max_rounds[i])
            
            '''

            # 네 번째 제약식: 최대 착지 수 제한과 주문 착지 방문 여부를 결합
            solver.Add(solver.Sum(visit[i, j, r, str(h)] for h in range(num_stops)) <= max_count[i])
            for k in range(order_last):
                h = str(order_stop[k])  # k번째 주문의 착지 h
                solver.Add(V[i, j, r, k] <= visit[i, j, r, h])
'''
            
            # 다섯 번째 제약식: 최대 적재량 및 적재 무게 제한
            solver.Add(solver.Sum(V[i, j, r, k] * order_plt[k] for k in range(order_last)) <= maxcapa_plt[i])
            solver.Add(solver.Sum(V[i, j, r, k] * order_weight[k] for k in range(order_last)) <= maxcapa_weight[i])

            # 여섯 번째 제약식: 접근 제한
            for h in range(num_stops):
                solver.Add(visit[i, j, r, str(h)] * Vton[i] <= stop_access[h])

            # 일곱 번째 제약식: 시간 제약
            if r == 0:
                Tdepart_ijr0 = solver.NumVar(0, solver.infinity(), f'Tdepart_{i}_{j}_{r}_0')
                solver.Add(Tdepart_ijr0 >= Tarrive[i, j, r, 0])
                solver.Add(Tdepart_ijr0 >= center_start)
                solver.Add(Tdepart[i, j, r, 0] == Tdepart_ijr0 + center_load_time)

            for k in range(1, order_last):
                if k >= len(order_stop):
                    break  # 인덱스 초과 방지
                h = order_stop[k - 1]
                h_next = order_stop[k]

                # 각 스톱에서의 언로드 시간
                solver.Add(Tdepart[i, j, r, k] == Tarrive[i, j, r, k] + stop_unload[int(h)] + stop_unloadplt[int(h)] * order_plt[k])

                # 노드 간 이동 시간
                origin = f'S_{h}'
                destination = f'S_{h_next}'
                if origin in od_time.index and destination in od_time.columns:
                    travel_time = od_time.loc[origin, destination]
                    solver.Add(Tarrive[i, j, r, k] == Tdepart[i, j, r, k - 1] + travel_time)

                # 스톱의 시작 및 종료 시간
                solver.Add(Tarrive[i, j, r, k - 1] >= stop_start[int(h)])
                solver.Add(Tarrive[i, j, r, k - 1] <= stop_end[int(h)])

                # 차량의 비즈니스 시작 및 종료 시간
                solver.Add(Tarrive[i, j, r, k - 1] >= Vstart[i])
                solver.Add(Tarrive[i, j, r, k - 1] <= Vend[i])
                solver.Add(Tdepart[i, j, r, k - 1] >= Vstart[i])
                solver.Add(Tdepart[i, j, r, k - 1] <= Vend[i])


            # 센터로 돌아오는 시간
            solver.Add(Tarrive[i, j, r, order_last-1] >= center_start)
            solver.Add(Tarrive[i, j, r, order_last-1] <= center_end)
    T.append(time.time())
    print(f'something happend in constraints -({j}) {T[j+1]-T[j]})')

print("all alright")


pass
something happend 0-(0) 1.283858299255371
something happend 0-(1) 1.2735941410064697
something happend 0-(2) 1.4085884094238281
something happend 0-(3) 1.1947753429412842
something happend 0-(4) 1.2022807598114014
t1~t2:  6.816402196884155


IndexError: index 1092 is out of bounds for axis 0 with size 1092

In [3]:
##dummy data 대신 들어가야할 것


for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        Vround_name = f'Vround_{i}_{j}'
        Vdistance_name = f'Vdistance_{i}_{j}'
        Vplt_name = f'Vplt_{i}_{j}'
        Vcarry_name = f'Vcarry_{i}_{j}'
        
        Vround[i, j] = solver.IntVar(0, max_rounds[i], Vround_name)
        Vdistance[i, j] = solver.NumVar(0, solver.infinity(), Vdistance_name)
        Vplt[i, j] = solver.NumVar(0, maxcapa_plt[i], Vplt_name)
        Vcarry[i, j] = solver.NumVar(0, maxcapa_weight[i], Vcarry_name)

        for r in range(max_rounds[i]):
            for k in range(order_last):
                V_name = f'V_{i}_{j}_{r}_{k}'
                Tarrive_name = f'Tarrive_{i}_{j}_{r}_{k}'
                Tdepart_name = f'Tdepart_{i}_{j}_{r}_{k}'

                V[i, j, r, k] = solver.BoolVar(V_name)
                Tarrive[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tarrive_name)
                Tdepart[i, j, r, k] = solver.NumVar(0, solver.infinity(), Tdepart_name)
            
            for h in range(len(stop_id)):
                visit_name = f'visit_{i}_{j}_{r}_{h}'
                visit[i, j, r, h] = solver.BoolVar(visit_name)          
    print(time.time())
    print(f'something happend 0-({i})')

t2 = time.time()
print('t1~t2: ',t2-t1)

KeyboardInterrupt: 

In [9]:
from ortools.linear_solver import pywraplp
import time

t1 = time.time()
T = []
print('pass')

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Example values (테스트를 위해 적절히 줄인 값들)
num_vehicle_types = 2  # 예제 차량 유형 수 (기존보다 줄인 값)
num_vehicles = 10  # 예제 차량 수 (기존보다 줄인 값)
order_last = 10  # 예제 마지막 주문 번호 (기존보다 줄인 값)
max_round = [3, 2]  # 예제 각 차량 유형의 최대 회전 수 (기존보다 줄인 값)
max_rounds = [int(max_round[i]) for i in range(num_vehicle_types)]  # 각 차량 유형의 최대 회전 수 (정수로 변환)
num_stops = 5  # 예제 stop 수 (기존보다 줄인 값)

# 더미 데이터 예제
Vton = [10, 20]  # 차량 유형별 최대 무게
max_count = [3, 4]  # 차량 유형별 최대 방문 수
order_stop = ['0', '1', '2', '3', '4', '0', '1', '2', '3', '4']  # 주문의 착지 ID (문자열로)
order_plt = [1] * order_last  # 주문별 적재량 (더미 데이터)
order_weight = [5] * order_last  # 주문별 무게 (더미 데이터)
maxcapa_plt = [15, 20]  # 차량 유형별 최대 적재량
maxcapa_weight = [100, 150]  # 차량 유형별 최대 적재 무게
stop_access = [15, 25, 35, 45, 55]  # 각 stop의 접근 제한 무게 (더미 데이터)

# 더미 변수 생성
V = {}
Vround = {}
Vdistance = {}
Vplt = {}
Vcarry = {}
visit = {}
Tarrive = {}
Tdepart = {}

for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        Vround[i, j] = solver.IntVar(0, max_rounds[i], f'Vround_{i}_{j}')
        Vdistance[i, j] = solver.NumVar(0, solver.infinity(), f'Vdistance_{i}_{j}')
        Vplt[i, j] = solver.NumVar(0, maxcapa_plt[i], f'Vplt_{i}_{j}')
        Vcarry[i, j] = solver.NumVar(0, maxcapa_weight[i], f'Vcarry_{i}_{j}')
        for r in range(max_rounds[i]):
            for k in range(order_last):
                V[i, j, r, k] = solver.BoolVar(f'V_{i}_{j}_{r}_{k}')
                Tarrive[i, j, r, k] = solver.NumVar(0, solver.infinity(), f'Tarrive_{i}_{j}_{r}_{k}')
                Tdepart[i, j, r, k] = solver.NumVar(0, solver.infinity(), f'Tdepart_{i}_{j}_{r}_{k}')
            for h in range(num_stops):
                visit[i, j, r, str(h)] = solver.BoolVar(f'visit_{i}_{j}_{r}_{h}')

t2 = time.time()
print('t1~t2: ', t2-t1)

# 첫 번째 제약식 추가
T = [t2]
for i in range(num_vehicle_types):
    for j in range(num_vehicles - 1):  # j+1 때문에 num_vehicles-1까지만
        lhs = solver.Sum(V[i, j+1, r, k] for r in range(max_rounds[i]) for k in range(order_last))
        rhs = solver.Sum(V[i, j, r, k] for r in range(max_rounds[i]) for k in range(order_last))
        solver.Add(lhs <= rhs)
    T.append(time.time())
    print(f'something happened 1-({i}): {T[i+1]-T[i]}')

t3 = time.time()
print('t2~t3: ', t3-t2)

M = 100000  # Sufficiently large number

# 두 번째 제약식 추가
T = [t3]
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        total_visits = solver.Sum(V[i, j, r, k] for r in range(max_rounds[i]) for k in range(order_last))
        solver.Add(Vround[i, j] <= M * total_visits)
        solver.Add(Vdistance[i, j] <= M * total_visits)
        solver.Add(Vplt[i, j] <= M * total_visits)
        solver.Add(Vcarry[i, j] <= M * total_visits)
    T.append(time.time())
    print(f'something happened 2-({i}): {T[i+1]-T[i]}')

t4 = time.time()
print('t3~t4: ', t4-t3)

# 세 번째 제약식 추가
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        solver.Add(Vround[i, j] <= max_rounds[i])

# 네 번째 제약식: 최대 착지 수 제한과 주문 착지 방문 여부를 결합
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            # 최대 착지 수 제한
            solver.Add(solver.Sum(visit[i, j, r, str(h)] for h in range(num_stops)) <= max_count[i])
            # 주문 착지 방문 여부
            for k in range(order_last):
                h = order_stop[k]  # k번째 주문의 착지 h
                solver.Add(V[i, j, r, k] <= visit[i, j, r, h])

# 다섯 번째 제약식 추가 : max capa plt, weight
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            solver.Add(solver.Sum(V[i, j, r, k] * order_plt[k] for k in range(order_last)) <= maxcapa_plt[i])
            solver.Add(solver.Sum(V[i, j, r, k] * order_weight[k] for k in range(order_last)) <= maxcapa_weight[i])

# 여섯 번째 제약식 추가 (1): 방문 여부를 결정
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            for k in range(order_last):
                h = order_stop[k]  # stop_id를 정수로 변환
                solver.Add(V[i, j, r, k] <= visit[i, j, r, h])

# 여섯 번째 제약식 추가 (2): 접근 제한
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            for h in range(num_stops):
                solver.Add(visit[i, j, r, str(h)] * Vton[i] <= stop_access[h])

# 일곱 번째 제약식 추가 : 
for i in range(num_vehicle_types):
    for j in range(num_vehicles):
        for r in range(max_rounds[i]):
            # 센터에서 로드 시간
            Tdepart_ijr0 = solver.NumVar(0, solver.infinity(), f'Tdepart_{i}_{j}_{r}_0')
            solver.Add(Tdepart_ijr0 >= Tarrive[i, j, r, 0])
            solver.Add(Tdepart_ijr0 >= center_start)
            solver.Add(Tdepart[i, j, r, 0] == Tdepart_ijr0 + center_load_time)

            for k in range(1, order_last):
                h = order_stop[k - 1]
                h_next = order_stop[k]
                
                # 각 스톱에서의 언로드 시간
                solver.Add(Tdepart[i, j, r, k] == Tarrive[i, j, r, k] + stop_unload[int(h)] + stop_unloadplt[int(h)] * order_plt[k])
                
                # 노드 간 이동 시간
                origin = f'S_{h}'
                destination = f'S_{h_next}'
                if origin in od_time.index and destination in od_time.columns:
                    travel_time = od_time.loc[origin, destination]
                    solver.Add(Tarrive[i, j, r, k] == Tdepart[i, j, r, k-1] + travel_time)
                
                # 스톱의 시작 및 종료 시간
                solver.Add(Tarrive[i, j, r, k-1] >= stop_start[int(h)])
                solver.Add(Tarrive[i, j, r, k-1] <= stop_end[int(h)])
                
                # 차량의 비즈니스 시작 및 종료 시간
                solver.Add(Tarrive[i, j, r, k-1] >= Vstart[i])
                solver.Add(Tarrive[i, j, r, k-1] <= Vend[i])
                solver.Add(Tdepart[i, j, r, k-1] >= Vstart[i])
                solver.Add(Tdepart[i, j, r, k-1] <= Vend[i])
            
            # 센터로 돌아오는 시간
            solver.Add(Tarrive[i, j, r, order_last-1] >= center_start)
            solver.Add(Tarrive[i, j, r, order_last-1] <= center_end)

print("all alright")


[1, 2, 3, 4]
[1]
